In [8]:
import pandas as pd
import sqlalchemy as sal

engine = sal.create_engine('mssql+pyodbc://siva\\SQLEXPRESS/namaste_sql_assignements?driver=ODBC+Driver+17+for+SQL+Server')
conn=engine.connect()


In [41]:
def extract():
    df_products=pd.read_csv("C:\\Users\\skesi\\Downloads\\product.txt")
    df_products_db=pd.read_sql_query("select * from products_dim where end_date = '9999-12-31'" , conn)
    
    return df_products,df_products_db
def transfrom(df_products,df_products_db):
    df_merged=pd.merge(df_products,df_products_db, how='inner', on='product_id')
    updated_rows=df_merged['product_key']
    keys = updated_rows.to_list()
    product_keys= ','.join([str(key) for key in keys])
    return product_keys
   
def insert(df_products):
    df_products['start_date'] = pd.to_datetime('now').strftime('%Y-%m-%d')
    df_products['end_date'] = '9999-12-31'
    df_products.to_sql('products_dim',conn,index=False,if_exists='append')
    conn.commit()

    
def updates(keys):
    if keys!=' ':
        query = sal.text("update products_dim set end_date =  cast(getdate()-1 as date) where product_key in (" + product_keys + ") ")
        p = conn.execute(query)
        conn.commit()
   

In [42]:
df_products,df_products_db=extract()
product_keys=transfrom(df_products,df_products_db)
insert(df_products)
updates(product_keys)

In [38]:
updated_rows=df_merged['product_key']
updated_rows.to_list()

[1]

In [19]:
keys = updated_rows.to_list()

[1]

In [20]:
df_products

,product_id,product_name,price
0,1,IPhone 11,35000
1,5,Laptop,65000


In [21]:
df_products_db

,product_key,product_id,product_name,price,start_date,end_date
0,1,1,IPhone 11,30000,2024-11-28,9999-12-31
1,2,4,ipad,30000,2024-11-28,9999-12-31


In [ ]:
update product_dim 
set product_dim.end_date=CURDATE()-1 
where products_dim.product_key in keys